In [4]:
import pandas as pd
import pathlib as pl
import collections as col

uw_file = pl.Path("hgsvc3_missing_accessions.hifi-ont.UW.tsv")
all_file = pl.Path("notebooks/hgsvc3_missing_accessions.hifi-ont.ALL.tsv").resolve(strict=True)

uw_table = pd.read_csv(uw_file, sep="\t", header=0)
all_table = pd.read_csv(all_file, sep="\t", header=0)

all_table["processed_by"] = "nobody"

uw_files = all_table["file_name"].isin(uw_table["file_name"])
all_table.loc[uw_files, "processed_by"] = "UW_YJK"

error_file_name = "20230626_230620_23-lee-007_PCA100115_3F-run15_guppy-5.0.11-sup-prom_fastq_pass"
error_select = all_table["file_name"] == error_file_name
all_table.loc[error_select, "processed_by"] = "ERROR_JAX_PH"

jax_file = "m64119e_230531_095655.hifi_reads"
jax_select = all_table["file_name"] == jax_file
all_table.loc[jax_select, "processed_by"] = "JAX_PH"

print(all_table["processed_by"].value_counts())
index_setters = col.defaultdict(list)
for row in all_table.loc[all_table["processed_by"] == "nobody", :].itertuples():
    if row.accession != "unknown":
        index_setters["external-data"].append(row.Index)
    elif "edevi" in row.file_name.lower():
        index_setters["UMIGS_SD"].append(row.Index)
    elif "clee" in row.file_name.lower():
        index_setters["JAX_PH"].append(row.Index)
    elif "eee" in row.file_name.lower():
        index_setters["UW_EICHLER"].append(row.Index)
        print(row.file_name)
    elif "hg00514" in row.remote_path.lower() and "reseq" in row.remote_path.lower():
        index_setters["UW_EICHLER"].append(row.Index)
        print(row.file_name)
    else:
        print(row)
        raise

for label, indices in index_setters.items():
    all_table.loc[indices, "processed_by"] = label

print(all_table["processed_by"].value_counts())
all_table.sort_values(["processed_by", "datatype", "remote_path", "file_name"], inplace=True)
all_table = all_table[
    ["sample", "datatype", "processed_by", "file_name", "remote_path", "accession"]
]
# with open("hgsvc3_missing-accessions.hifi-ont.all.tsv", "w") as dump:
#     all_table.to_csv(dump, sep="\t", header=True, index=False)
    

processed_by
UW_YJK          285
nobody           45
JAX_PH            1
ERROR_JAX_PH      1
Name: count, dtype: int64
m54329U_200719_061020.ccs
m54329U_200717_235548.ccs
m54329U_200715_194535.ccs
processed_by
UW_YJK           285
UMIGS_SD          27
JAX_PH            14
UW_EICHLER         3
external-data      2
ERROR_JAX_PH       1
Name: count, dtype: int64
